In [3]:
###
### Libraries
###

import pymongo

In [5]:
###
### Create a client object connected locally
###

client = pymongo.MongoClient('localhost', 27017)

In [21]:
###
### Create a database for our client to connect to, if not exists, create it
###

###
### Drop a database
###
# client.drop_database('chapter3')

db = client['UCI-Database']

In [7]:
collection = db['income']

In [22]:
###
### Pro-tip: Documents have a JSON structure, i.e,.. a python {} structure of key-value pairs
### This means we can leverage the zip command for each line, zipping values by location with the right
###   `column_names` value, then calling that a dict. This is the `document` that MongoDB is expecting
###

column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 
               'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain',
               'capital-loss', 'hours-per-week', 'native-country', 'label']

###
### Dataset location: https://archive.ics.uci.edu/ml/datasets/adult
### Just download it and save it, the column names are on the homepage also.
###

row_container=list()
with open("income.txt") as f_in:
    for line in f_in:
        row = [val.strip(" ").strip("\n") for val in line.split(',')]
        row_dict = dict(zip(column_names, row))
        collection.insert_one(row_dict)
        row_container.append(row_dict) # store it locally for debugging

In [23]:
###
### Verify things were added successfully
###

collection.count()

32562